In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [13]:
from keras.models import Model, Sequential
from keras.layers import Activation, Dense, BatchNormalization, Dropout, Conv2D, concatenate, Conv2DTranspose, MaxPooling2D, UpSampling2D, Input, Reshape
from keras import backend as K
from keras.layers.core import SpatialDropout2D
from keras.optimizers import Adam, SGD
import tensorflow as tf
from tensorflow.keras.utils import plot_model
import numpy as np
import pandas as pd
import glob
import PIL
from PIL import Image
from PIL import ImageFilter, ImageOps, ImageEnhance
import matplotlib.pyplot as plt
import cv2
from keras.layers import Lambda
%matplotlib inline

from sklearn.model_selection import train_test_split
from warnings import filterwarnings

filterwarnings('ignore')
plt.rcParams["axes.grid"] = False
np.random.seed(101)



In [3]:
import re
numbers = re.compile(r'(\d+)')
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts


In [4]:
filelist_trainx_ph2 = sorted(glob.glob('/content/drive/MyDrive/ISCAN/x_train/*.png'), key=numericalSort)
X_train_ph2 = np.array([np.array(Image.open(fname)) for fname in filelist_trainx_ph2])

filelist_trainy_ph2 = sorted(glob.glob('/content/drive/MyDrive/ISCAN/y_train/*.png'), key=numericalSort)
Y_train_ph2 = np.array([np.array(Image.open(fname)) for fname in filelist_trainy_ph2])

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
plt.imshow(X_train_ph2[0])
plt.xlabel("Dimensions: "+str(np.array(X_train_ph2[0]).shape))
plt.subplot(1,2,2)
plt.imshow(Y_train_ph2[0], plt.cm.binary_r)
plt.xlabel("Dimensions: "+str(np.array(Y_train_ph2[0]).shape))
plt.show()

In [6]:
x_train, x_test, y_train, y_test = train_test_split(X_train_ph2, Y_train_ph2, test_size = 0.25, random_state = 101)

In [7]:
def jaccard_distance(y_true, y_pred, smooth=100):
    #y_true = tf.dtypes.cast(y_true, tf.int32)
    #y_pred = tf.dtypes.cast(y_pred,tf.int32)
    #y_true = tf.cast(y_true, tf.float32)
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    sum_ = K.sum(K.square(y_true), axis = -1) + K.sum(K.square(y_pred), axis=-1)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return (1 - jac)

def iou(y_true, y_pred, smooth = 100):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    sum_ = K.sum(K.square(y_true), axis = -1) + K.sum(K.square(y_pred), axis=-1)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return jac

def dice_coef(y_true, y_pred, smooth = 100):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def precision(y_true, y_pred):
    '''Calculates the precision, a metric for multi-label classification of
    how many selected items are relevant.
    '''
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    '''Calculates the recall, a metric for multi-label classification of
    how many relevant items are selected.
    '''
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def accuracy(y_true, y_pred):
    '''Calculates the mean accuracy rate across all predictions for binary
    classification problems.
    '''
    return K.mean(K.equal(y_true, K.round(y_pred)))

In [42]:
def random_rotation(x_image, y_image):
    rows_x,cols_x, chl_x = x_image.shape
    rows_y,cols_y = y_image.shape
    rand_num = np.random.randint(-40,40)
    M1 = cv2.getRotationMatrix2D((cols_x/2,rows_x/2),rand_num,1)
    M2 = cv2.getRotationMatrix2D((cols_y/2,rows_y/2),rand_num,1)
    x_image = cv2.warpAffine(x_image,M1,(cols_x,rows_x))
    y_image = cv2.warpAffine(y_image.astype('float32'),M2,(cols_y,rows_y))
    return np.array(x_image), np.array(y_image.astype('int'))

def horizontal_flip(x_image, y_image):
    x_image = cv2.flip(x_image, 1)
    y_image = cv2.flip(y_image.astype('float32'), 1)
    return x_image, y_image.astype('int')

def vertical_flip(x_image, y_image):
    x_image = cv2.flip(x_image, 0)
    y_image = cv2.flip(y_image.astype('float32'), 0)
    return x_image, y_image.astype('int')

def horizontal_vertical_flip(x_image, y_image):
    x_image = cv2.flip(x_image, -1)
    y_image = cv2.flip(y_image.astype('float32'), -1)
    return x_image, y_image.astype('int')

def gaussian_blur(img, kernel_size=3):
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)


def shear(img, shear_factor=0.2):
    img = Image.fromarray(img)
    width, height = img.size
    xshift = int(shear_factor * width)
    transform_matrix = (1, -shear_factor, xshift, 0, 1, 0)
    return img.transform((width + xshift, height), Image.AFFINE, transform_matrix, Image.BICUBIC)



In [43]:
def img_augmentation(x_train, y_train):
    x_rotat = []
    y_rotat = []
    x_flip = []
    y_flip = []
    x_v_flip = []
    y_v_flip = []
    x_hv_flip = []
    y_hv_flip = []
    x_gaussian_blur = []
    y_gaussian_blur = []
    x_shear = []
    y_shear = []
    for idx in range(len(x_train)):

        # Apply Random Rotation
        x,y = random_rotation(x_train[idx], y_train[idx])
        x_rotat.append(x)
        y_rotat.append(y)
        
        # Apply Horizintal Flip
        x,y = horizontal_flip(x_train[idx], y_train[idx])
        x_flip.append(x)
        y_flip.append(y)

        # Apply Vertical Flip
        x,y = vertical_flip(x_train[idx], y_train[idx])
        x_v_flip.append(x)
        y_v_flip.append(y)

        # Apply Horizintal Vertical Flip
        x,y = horizontal_vertical_flip(x_train[idx], y_train[idx])
        x_hv_flip.append(x)
        y_hv_flip.append(y)

        # Apply Gaussian blur
        x, y = gaussian_blur(x_train[idx]), y_train[idx]
        x_gaussian_blur.append(x)
        y_gaussian_blur.append(y)
    
        # Apply shear
        x, y = shear(x_train[idx]), y_train[idx]
        x_shear.append(x)
        y_shear.append(y)
        
    return np.array(x_rotat), np.array(y_rotat), np.array(x_flip), np.array(y_flip), np.array(x_v_flip), np.array(y_v_flip), np.array(x_hv_flip), np.array(y_hv_flip), np.array(x_gaussian_blur), np.array(y_gaussian_blur), np.array(x_shear), np.array(y_shear)

In [44]:
def img_augmentation(x_test, y_test):
    x_rotat = []
    y_rotat = []
    x_flip = []
    y_flip = []
    x_v_flip = []
    y_v_flip = []
    x_hv_flip = []
    y_hv_flip = []
    x_gaussian_blur = []
    y_gaussian_blur = []
    x_shear = []
    y_shear = []
    for idx in range(len(x_test)):
        x,y = random_rotation(x_test[idx], y_test[idx])
        x_rotat.append(x)
        y_rotat.append(y)
        
        x,y = horizontal_flip(x_test[idx], y_test[idx])
        x_flip.append(x)
        y_flip.append(y)

        x,y = vertical_flip(x_test[idx], y_test[idx])
        x_v_flip.append(x)
        y_v_flip.append(y)

        x,y = horizontal_vertical_flip(x_test[idx], y_test[idx])
        x_hv_flip.append(x)
        y_hv_flip.append(y)

        # Apply Gaussian blur
        x, y = gaussian_blur(x_test[idx]), y_test[idx]
        x_gaussian_blur.append(x)
        y_gaussian_blur.append(y)
    
        # Apply shear
        x, y = shear(x_test[idx]), y_test[idx]
        x_shear.append(x)
        y_shear.append(y)
        
    return np.array(x_rotat), np.array(y_rotat), np.array(x_flip), np.array(y_flip), np.array(x_v_flip), np.array(y_v_flip), np.array(x_hv_flip), np.array(y_hv_flip), np.array(x_gaussian_blur), np.array(y_gaussian_blur), np.array(x_shear), np.array(y_shear)

In [45]:
x_rotated, y_rotated, x_flipped, y_flipped, x_v_flipped, y_v_flipped, x_hv_flipped, y_hv_flipped, x_gaussian_blur, y_gaussian_blur, x_shear, y_shear = img_augmentation(x_train, y_train)
x_rotated_t, y_rotated_t, x_flipped_t, y_flipped_t, x_v_flipped_t, y_v_flipped_t, x_hv_flipped_t, y_hv_flipped_t, x_gaussian_blur_t, y_gaussian_blur_t, x_shear_t, y_shear_t = img_augmentation(x_test, y_test)

In [ ]:
img_num = 112
plt.figure(figsize=(12,12))
plt.subplot(7,2,1)
plt.imshow(x_train[img_num])
plt.title('Original Image')
plt.subplot(7,2,2)
plt.imshow(y_train[img_num], plt.cm.binary_r)
plt.title('Original Mask')
plt.subplot(7,2,3)
plt.imshow(x_rotated[img_num])
plt.title('Rotated Image')
plt.subplot(7,2,4)
plt.imshow(y_rotated[img_num], plt.cm.binary_r)
plt.title('Rotated Mask')
plt.subplot(7,2,5)
plt.imshow(x_flipped[img_num])
plt.title('Horizontal Flipped Image')
plt.subplot(7,2,6)
plt.imshow(y_flipped[img_num], plt.cm.binary_r)
plt.title('Horizontal Flipped Mask')
plt.subplot(7,2,7)
plt.imshow(x_v_flipped[img_num], plt.cm.binary_r)
plt.title('Vertical Flipped Image')
plt.subplot(7,2,8)
plt.imshow(y_v_flipped[img_num], plt.cm.binary_r)
plt.title('Vertical Flipped Mask')
plt.subplot(7,2,9)
plt.imshow(x_hv_flipped[img_num], plt.cm.binary_r)
plt.title('Horizontal-Vertical Flipped Image')
plt.subplot(7,2,10)
plt.imshow(y_hv_flipped[img_num], plt.cm.binary_r)
plt.title('Horizontal-Vertical Flipped Mask')
plt.subplot(7,2,11)
plt.imshow(x_gaussian_blur[img_num], plt.cm.binary_r)
plt.title('Gaussian Blur Image')
plt.subplot(7,2,12)
plt.imshow(y_gaussian_blur[img_num], plt.cm.binary_r)
plt.title('Gaussian Blur Mask')
plt.subplot(7,2,13)
plt.imshow(x_shear[img_num], plt.cm.binary_r)
plt.title('Sheared Image')
plt.subplot(7,2,14)
plt.imshow(y_shear[img_num], plt.cm.binary_r)
plt.title('Sheared Mask')
plt.show()

In [48]:
# For training Set
x_train_full = np.concatenate([x_train, x_rotated, x_flipped, x_v_flipped, x_hv_flipped, x_gaussian_blur]) # x_shear])
y_train_full = np.concatenate([y_train, y_rotated, y_flipped, y_v_flipped, y_hv_flipped, y_gaussian_blur]) # y_shear])

In [49]:
x_train, x_val, y_train, y_val = train_test_split(x_train_full, y_train_full, test_size = 0.20, random_state = 101)

In [ ]:
print("Length of the Training Set   : {}".format(len(x_train)))
print("Length of the Test Set       : {}".format(len(x_test)))
print("Length of the Validation Set : {}".format(len(x_val)))

In [18]:

# Number of image channels (for example 3 in case of RGB, or 1 for grayscale images)
INPUT_CHANNELS = 3
# Number of output masks (1 in case you predict only one type of objects)
OUTPUT_MASK_CHANNELS = 1
# Pretrained weights

In [19]:
def segnet(epochs_num,savename):

    # Encoding layer
    img_input = Input(shape= (192, 256, 3))
    x = Lambda(lambda image: image / 255.0)(img_input)  # normalization added
    x = Conv2D(64, (3, 3), padding='same', name='conv1',strides= (1,1))(x)
    x = BatchNormalization(name='bn1')(x)
    x = Activation('relu')(x)
    x = Conv2D(64, (3, 3), padding='same', name='conv2')(x)
    x = BatchNormalization(name='bn2')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D()(x)
    
    x = Conv2D(128, (3, 3), padding='same', name='conv3')(x)
    x = BatchNormalization(name='bn3')(x)
    x = Activation('relu')(x)
    x = Conv2D(128, (3, 3), padding='same', name='conv4')(x)
    x = BatchNormalization(name='bn4')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D()(x)

    x = Conv2D(256, (3, 3), padding='same', name='conv5')(x)
    x = BatchNormalization(name='bn5')(x)
    x = Activation('relu')(x)
    x = Conv2D(256, (3, 3), padding='same', name='conv6')(x)
    x = BatchNormalization(name='bn6')(x)
    x = Activation('relu')(x)
    x = Conv2D(256, (3, 3), padding='same', name='conv7')(x)
    x = BatchNormalization(name='bn7')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D()(x)

    x = Conv2D(512, (3, 3), padding='same', name='conv8')(x)
    x = BatchNormalization(name='bn8')(x)
    x = Activation('relu')(x)
    x = Conv2D(512, (3, 3), padding='same', name='conv9')(x)
    x = BatchNormalization(name='bn9')(x)
    x = Activation('relu')(x)
    x = Conv2D(512, (3, 3), padding='same', name='conv10')(x)
    x = BatchNormalization(name='bn10')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D()(x)
    
    x = Conv2D(512, (3, 3), padding='same', name='conv11')(x)
    x = BatchNormalization(name='bn11')(x)
    x = Activation('relu')(x)
    x = Conv2D(512, (3, 3), padding='same', name='conv12')(x)
    x = BatchNormalization(name='bn12')(x)
    x = Activation('relu')(x)
    x = Conv2D(512, (3, 3), padding='same', name='conv13')(x)
    x = BatchNormalization(name='bn13')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D()(x)

    x = Dense(1024, activation = 'relu', name='fc1')(x)
    x = Dense(1024, activation = 'relu', name='fc2')(x)
    # Decoding Layer 
    x = UpSampling2D()(x)
    x = Conv2DTranspose(512, (3, 3), padding='same', name='deconv1')(x)
    x = BatchNormalization(name='bn14')(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(512, (3, 3), padding='same', name='deconv2')(x)
    x = BatchNormalization(name='bn15')(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(512, (3, 3), padding='same', name='deconv3')(x)
    x = BatchNormalization(name='bn16')(x)
    x = Activation('relu')(x)
    
    x = UpSampling2D()(x)
    x = Conv2DTranspose(512, (3, 3), padding='same', name='deconv4')(x)
    x = BatchNormalization(name='bn17')(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(512, (3, 3), padding='same', name='deconv5')(x)
    x = BatchNormalization(name='bn18')(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(256, (3, 3), padding='same', name='deconv6')(x)
    x = BatchNormalization(name='bn19')(x)
    x = Activation('relu')(x)

    x = UpSampling2D()(x)
    x = Conv2DTranspose(256, (3, 3), padding='same', name='deconv7')(x)
    x = BatchNormalization(name='bn20')(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(256, (3, 3), padding='same', name='deconv8')(x)
    x = BatchNormalization(name='bn21')(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(128, (3, 3), padding='same', name='deconv9')(x)
    x = BatchNormalization(name='bn22')(x)
    x = Activation('relu')(x)

    x = UpSampling2D()(x)
    x = Conv2DTranspose(128, (3, 3), padding='same', name='deconv10')(x)
    x = BatchNormalization(name='bn23')(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(64, (3, 3), padding='same', name='deconv11')(x)
    x = BatchNormalization(name='bn24')(x)
    x = Activation('relu')(x)
    
    x = UpSampling2D()(x)
    x = Conv2DTranspose(64, (3, 3), padding='same', name='deconv12')(x)
    x = BatchNormalization(name='bn25')(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(1, (3, 3), padding='same', name='deconv13')(x)
    x = BatchNormalization(name='bn26')(x)
    x = Activation('sigmoid')(x)
    pred = Reshape((192,256))(x)

    
    model = Model(inputs=img_input, outputs=pred)
    
    model.compile(optimizer= SGD(lr=0.001, momentum=0.9, decay=0.0005, nesterov=False), loss= ["binary_crossentropy"]
                  , metrics=[iou, dice_coef, precision, recall, accuracy])
    model.summary()


    hist = model.fit(x_train, y_train, epochs= epochs_num, batch_size= 18, validation_data= (x_val, y_val), verbose=1)
    
    model.save(savename)
    return model,hist

In [ ]:
model_2, hist_2 = segnet(200, 'segnet_100_epoch.h5')

In [ ]:
plt.figure(figsize=(14, 35))
plt.suptitle('Training Statistics on Train Set')
plt.subplot(8,2,1)
plt.plot(hist_2.history['loss'], 'red')
plt.title('Training Loss for SegNet')
plt.subplot(8,2,2)
plt.plot(hist_2.history['accuracy'], 'green')
plt.title('Training Accuracy for SegNet')
plt.subplot(8,2,3)
plt.plot(hist_2.history['val_loss'], 'red')
plt.yticks(list(np.arange(0.0, 1.0, 0.10)))
plt.title('Valdiation Loss for SegNet')
plt.subplot(8,2,4)
plt.plot(hist_2.history['val_accuracy'], 'green')
plt.yticks(list(np.arange(0.0, 1.0, 0.10)))
plt.title('Validation Accuracy for SegNet')
plt.show()

In [22]:
img_input = Input(shape= (192, 256, 3))
x = Conv2D(64, (3, 3), padding='same', name='conv1',strides= (1,1))(img_input)
x = BatchNormalization(name='bn1')(x)
x = Activation('relu')(x)
x = Conv2D(64, (3, 3), padding='same', name='conv2')(x)
x = BatchNormalization(name='bn2')(x)
x = Activation('relu')(x)
x = MaxPooling2D()(x)

x = Conv2D(128, (3, 3), padding='same', name='conv3')(x)
x = BatchNormalization(name='bn3')(x)
x = Activation('relu')(x)
x = Conv2D(128, (3, 3), padding='same', name='conv4')(x)
x = BatchNormalization(name='bn4')(x)
x = Activation('relu')(x)
x = MaxPooling2D()(x)

x = Conv2D(256, (3, 3), padding='same', name='conv5')(x)
x = BatchNormalization(name='bn5')(x)
x = Activation('relu')(x)
x = Conv2D(256, (3, 3), padding='same', name='conv6')(x)
x = BatchNormalization(name='bn6')(x)
x = Activation('relu')(x)
x = Conv2D(256, (3, 3), padding='same', name='conv7')(x)
x = BatchNormalization(name='bn7')(x)
x = Activation('relu')(x)
x = MaxPooling2D()(x)

x = Conv2D(512, (3, 3), padding='same', name='conv8')(x)
x = BatchNormalization(name='bn8')(x)
x = Activation('relu')(x)
x = Conv2D(512, (3, 3), padding='same', name='conv9')(x)
x = BatchNormalization(name='bn9')(x)
x = Activation('relu')(x)
x = Conv2D(512, (3, 3), padding='same', name='conv10')(x)
x = BatchNormalization(name='bn10')(x)
x = Activation('relu')(x)
x = MaxPooling2D()(x)

x = Conv2D(512, (3, 3), padding='same', name='conv11')(x)
x = BatchNormalization(name='bn11')(x)
x = Activation('relu')(x)
x = Conv2D(512, (3, 3), padding='same', name='conv12')(x)
x = BatchNormalization(name='bn12')(x)
x = Activation('relu')(x)
x = Conv2D(512, (3, 3), padding='same', name='conv13')(x)
x = BatchNormalization(name='bn13')(x)
x = Activation('relu')(x)
x = MaxPooling2D()(x)

x = Dense(1024, activation = 'relu', name='fc1')(x)
x = Dense(1024, activation = 'relu', name='fc2')(x)
# Decoding Layer 
x = UpSampling2D()(x)
x = Conv2DTranspose(512, (3, 3), padding='same', name='deconv1')(x)
x = BatchNormalization(name='bn14')(x)
x = Activation('relu')(x)
x = Conv2DTranspose(512, (3, 3), padding='same', name='deconv2')(x)
x = BatchNormalization(name='bn15')(x)
x = Activation('relu')(x)
x = Conv2DTranspose(512, (3, 3), padding='same', name='deconv3')(x)
x = BatchNormalization(name='bn16')(x)
x = Activation('relu')(x)

x = UpSampling2D()(x)
x = Conv2DTranspose(512, (3, 3), padding='same', name='deconv4')(x)
x = BatchNormalization(name='bn17')(x)
x = Activation('relu')(x)
x = Conv2DTranspose(512, (3, 3), padding='same', name='deconv5')(x)
x = BatchNormalization(name='bn18')(x)
x = Activation('relu')(x)
x = Conv2DTranspose(256, (3, 3), padding='same', name='deconv6')(x)
x = BatchNormalization(name='bn19')(x)
x = Activation('relu')(x)

x = UpSampling2D()(x)
x = Conv2DTranspose(256, (3, 3), padding='same', name='deconv7')(x)
x = BatchNormalization(name='bn20')(x)
x = Activation('relu')(x)
x = Conv2DTranspose(256, (3, 3), padding='same', name='deconv8')(x)
x = BatchNormalization(name='bn21')(x)
x = Activation('relu')(x)
x = Conv2DTranspose(128, (3, 3), padding='same', name='deconv9')(x)
x = BatchNormalization(name='bn22')(x)
x = Activation('relu')(x)

x = UpSampling2D()(x)
x = Conv2DTranspose(128, (3, 3), padding='same', name='deconv10')(x)
x = BatchNormalization(name='bn23')(x)
x = Activation('relu')(x)
x = Conv2DTranspose(64, (3, 3), padding='same', name='deconv11')(x)
x = BatchNormalization(name='bn24')(x)
x = Activation('relu')(x)

x = UpSampling2D()(x)
x = Conv2DTranspose(64, (3, 3), padding='same', name='deconv12')(x)
x = BatchNormalization(name='bn25')(x)
x = Activation('relu')(x)
x = Conv2DTranspose(1, (3, 3), padding='same', name='deconv13')(x)
x = BatchNormalization(name='bn26')(x)
x = Activation('sigmoid')(x)
pred = Reshape((192,256))(x)

In [23]:
model_segnet = Model(inputs=img_input, outputs=pred)

model_segnet.compile(optimizer= SGD(lr=0.001, momentum=0.9, decay=0.0005, nesterov=False), loss= ["binary_crossentropy"]
              , metrics=[iou, dice_coef, precision, recall, accuracy])

In [24]:
model_segnet.load_weights('segnet_100_epoch.h5')

In [ ]:
print ( '\n ˜˜˜˜˜˜˜˜˜˜˜˜˜˜˜ Stats after 100 epoch ˜˜˜˜˜˜˜˜˜˜˜˜˜˜˜˜˜˜˜' )
print ( '\n−−−−−−−−−−−−−On Train Set−−−−−−−−−−−−−−−−−−−−−−−−−−\n' )
res = model_segnet.evaluate (x_train , y_train , batch_size = 18)
print('--------------------')
print('IOU:      | {:.2f} |'.format(res[1]))
print('Dice Coef:| {:.2f} |'.format(res[2]))
print('Precision:| {:.2f} |'.format(res[3]))
print('Recall:   | {:.2f} |'.format(res[4]))
print('Accuracy: | {:.2f} |'.format(res[5]*100))
print("Loss:     | {:.2f} |".format(res[0]))
print('--------------------')
print('\n_____________On Test Set____________________\n')
res = model_segnet.evaluate(x_test, y_test, batch_size= 18)
print('-------------------------')
print('IOU:      | {:.2f} |'.format(res[1]))
print('Dice Coef:| {:.2f} |'.format(res[2]))
print('Precision:| {:.2f} |'.format(res[3]))
print('Recall:   | {:.2f} |'.format(res[4]))
print('Accuracy: | {:.2f} |'.format(res[5]*100))
print("Loss:     | {:.2f} |".format(res[0]))
print('--------------------')
print('\n_____________On validation Set____________________\n')
res = model_segnet.evaluate(x_val, y_val, batch_size=18)
print('-------------------------')
print('IOU:      | {:.2f} |'.format(res[1]))
print('Dice Coef:| {:.2f} |'.format(res[2]))
print('Precision:| {:.2f} |'.format(res[3]))
print('Recall:   | {:.2f} |'.format(res[4]))
print('Accuracy: | {:.2f} |'.format(res[5]*100))
print("Loss:     | {:.2f} |".format(res[0]))
print('-------------------------')

plt.figure(figsize=(20,14))
plt.suptitle('Traning Statistics on Train Set')
plt.subplot(2,2,1)
plt.plot(hist_2.history['loss'], 'red')
plt.title('Training Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.subplot(2,2,2)
plt.plot(hist_2.history['accuracy'], 'green')
plt.title('Training Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.subplot(2,2,3)
plt.plot(hist_2.history['val_loss'], 'red')
plt.yticks(list(np.arange(0.0, 1.0, 0.10)))
plt.title('Validation Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.subplot(2,2,4)
plt.plot(hist_2.history['val_accuracy'], 'green')
plt.yticks(list(np.arange(0.0, 1.0, 0.10)))
plt.title('Validation Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.show()

img_num=1
img_pred = model_segnet.predict(x_test[img_num].reshape(1,192,256,3))
plt.figure(figsize =(16,16))
plt.subplot(1,3,1)
plt.imshow(x_test[img_num])
plt.title('Original Image')
plt.subplot(1,3,2)
plt.imshow(y_test[img_num], plt.cm.binary_r)
plt.title('Ground Truth')
plt.subplot(1,3,3)
plt.imshow(img_pred.reshape(192,256), plt.cm.binary_r)
plt.title('Predicted Output')
plt.show()

img_num=2
img_pred = model_segnet.predict(x_test[img_num].reshape(1,192,256,3))
plt.figure(figsize =(16,16))
plt.subplot(1,3,1)
plt.imshow(x_test[img_num])
plt.title('Original Image')
plt.subplot(1,3,2)
plt.imshow(y_test[img_num], plt.cm.binary_r)
plt.title('Ground Truth')
plt.subplot(1,3,3)
plt.imshow(img_pred.reshape(192,256), plt.cm.binary_r)
plt.title('Predicted Output')
plt.show()

img_num=0
img_pred = model_segnet.predict(x_test[img_num].reshape(1,192,256,3))
plt.figure(figsize =(16,16))
plt.subplot(1,3,1)
plt.imshow(x_test[img_num])
plt.title('Original Image')
plt.subplot(1,3,2)
plt.imshow(y_test[img_num], plt.cm.binary_r)
plt.title('Ground Truth')
plt.subplot(1,3,3)
plt.imshow(img_pred.reshape(192,256), plt.cm.binary_r)
plt.title('Predicted Output')
plt.show()

img_num=0
img_pred = model_segnet.predict(x_test[img_num].reshape(1,192,256,3))
plt.figure(figsize =(16,16))
plt.subplot(1,3,1)
plt.imshow(x_test[img_num])
plt.title('Original Image')
plt.subplot(1,3,2)
plt.imshow(y_test[img_num], plt.cm.binary_r)
plt.title('Ground Truth')
plt.subplot(1,3,3)
plt.imshow(img_pred.reshape(192,256), plt.cm.binary_r)
plt.title('Predicted Output')
plt.show()

def enhance(img):
    sub= (model_segnet.predict(img.reshape(1,192,256,3))).flatten()

    for i in range(len(sub)):
      if sub[i]>0.5:
          sub[i] = 1
      else:
          sub[i] = 0
    return sub


plt.figure(figsize=(12,12))
plt.suptitle('Comparing the Prediction after enhancement')
plt.subplot(3,2,1)
plt.imshow(y_test[1], plt.cm.binary_r)
plt.title('Ground Truth')
plt.subplot(3,2,2)
plt.imshow(enhance(x_test[1]).reshape(192,256), plt.cm.binary_r)
plt.title('Predicted')
plt.subplot(3,2,3)
plt.imshow(y_test[2], plt.cm.binary_r)
plt.title('Ground Truth')
plt.subplot(3,2,4)
plt.imshow(enhance(x_test[2]).reshape(192,256), plt.cm.binary_r)
plt.title('Predicted')
plt.subplot(3,2,5)
plt.imshow(y_test[3], plt.cm.binary_r)
plt.title('Ground Truth')
plt.subplot(3,2,6)
plt.imshow(enhance(x_test[3]).reshape(192,256), plt.cm.binary_r)
plt.title('Predicted')
plt.show()

def applyMask(img_num):
    img = x_test[img_num]
    mask = np.array(enhance(img).reshape(192,256), dtype=np.uint8)
    res = cv2.bitwise_and(img, img, mask = mask)

    return res

img_num = 2
masked_img = applyMask(img_num)
plt.figure(figsize =(16,16))
plt.subplot(1,3,1)
plt.imshow(x_test[img_num])
plt.title('Orginal Image')
plt.subplot(1,3,2)
plt.imshow(enhance(x_test[img_num]).reshape(192,256), plt.cm.binary_r)
plt.title('Predicted Mask')
plt.subplot(1,3,3)
plt.imshow(masked_img, plt.cm.binary_r)
plt.title('Segmented Output')
plt.show()